In [ ]:
'''
This notebook contains the code to merge all male data across the years based on sections.
There are datasets for 2012, 2012-1.5, 2013 and 2014
The different sections that will be merged are as follows:
**2012**
1. Cover
2. Roaster
3. Section 1: Education (All men 18 and above)
4. Section 2: Agriculture
5. Section 3: Assets
6. Section 4: Consumption and Expenditure
7. Section 5: Credit
8. Section 6: Employment and Income
9. Section 7: Economic Events/Shocks
10. Section 8: Community Participation and Social Network Membership

**2013**
1. Cover
2. Roaster
3. Section 1: Education: Males 19 years and older
4. Section 2: Agriculture
5. Section 3: Assets
6. Section 4: Consumption and Expenditure
7. Section 5: Credit
8. Section 6: Employment and Income
9. Section 7: Health
10. Section 8: Political Participation and Governance

**2014**
1. Cover
2. Roaster
3. Section 1: Education: Males 19 years and older
4. Section 2: Agriculture
5. Section 3: Assets
6. Section 4: Consumption and Expenditure
7. Section 5: Credit
8. Section 6: Employment and Income
9. Section 7: Economic Events/Shocks”
10. Section 8: Participation in Social Safety Net
11. Section 9: Siblings
12. Section 10: Transfers
13. Section 11: Health and Nutrition

'''

In [1]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np
# Store excel file locations to variables
EI_2012 = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\29. Section 6 Part 6  Other income\2012_s6p6_m.xlsx"
EI_2012_5 = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\29. Section 6 Part 6  Other income\2012_5_s7p5.xlsx"
EI_2013 = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\29. Section 6 Part 6  Other income\2013_s6p5_m.xlsx"
EI_2014 = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\29. Section 6 Part 6  Other income\2014_s6p5_m.xlsx"

# Read excel files 
df_2012 = pd.read_excel(EI_2012)
df_2012_5 = pd.read_excel(EI_2012_5)
df_2013 = pd.read_excel(EI_2013)
df_2014 = pd.read_excel(EI_2014)



In [2]:
#This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.

# Rename columns in df to df_2014 column names


df_2012.rename(columns={
    'income_id': 's6p5_sr',
    'S6P6Q1': 's6p5_q2',
    'S6P6Q2': 's6p5_q3',
    'S6P6Q3': 's6p5_q4',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'MAUZA_ID': 'M_ID'
    
}, inplace=True)

df_2012_5.rename(columns={
    'Round': 'round',
    'S7P5SOURCE': 's6p5_sr',
    'S7P5_NAME': 's6p5_source',
    'S7P5Q2': 's6p5_q1',
    'S7P5Q3': 's6p5_q2',
    'S7P5Q4': 's6p5_q3',
    'S7P5Q5': 's6p5_q4',
    'C_PROVINCE': 'P_ID',
    'C_DISTRICT': 'D_ID',
    'C_TEHSIL': 'T_ID',
    'C_UC': 'UC_ID',
    'C_MOUZA': 'M_ID'
    
}, inplace=True)


# df_2014 doesn't need renaming as it is the reference

In [3]:
# Define the column mappings based on the provided positions
# Make dictionaries for each year with updated names
# replace with 'None' where there are no columns
#Here are the updated mapping lists for the given datasets:


mapping_2012 = [
    'hid', 'round', 's6p5_sr', None, None, 's6p5_q2', 's6p5_q3', 's6p5_q4', 'P_ID', 
    'D_ID', 'T_ID', 'UC_ID', 'M_ID', None
]



mapping_2012_5 = [
    'hid', 'round', 's6p5_sr', 's6p5_source', 's6p5_q1', 's6p5_q2', 
    's6p5_q3', 's6p5_q4', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID', 'C_HH_NUM'
]

mapping_2013 = [
    'hid', 'round', 's6p5_sr', 's6p5_source', 's6p5_q1', 's6p5_q2', 
    's6p5_q3', 's6p5_q4', None, None, None, None, None, None
]

mapping_2014 = [
    'hid', 'round', 's6p5_sr', 's6p5_source', 's6p5_q1', 's6p5_q2', 
    's6p5_q3', 's6p5_q4', None, None, None, None, None, None
]





In [4]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col) 
for col in mapping_2012_5:
    if col and col not in all_columns:
        all_columns.append(col)     
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [5]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [6]:
# Usage with dataframes and mappings
dfs = [ df_2012, df_2012_5, df_2013, df_2014]
mappings = [ mapping_2012, mapping_2012_5, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'hid', 'round', 's6p5_sr', 's6p5_q2', 's6p5_q3', 's6p5_q4', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column hid
Appending data for column round
Appending data for column s6p5_sr
Appending data for column s6p5_q2
Appending data for column s6p5_q3
Appending data for column s6p5_q4
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_ID
Appending data for column M_ID
Processing DataFrame with columns: ['Unnamed: 0', 'round', 'hid', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID', 'C_HH_NUM', 's6p5_sr', 's6p5_source', 's6p5_q1', 's6p5_q2', 's6p5_q3', 's6p5_q4']
Appending data for column hid
Appending data for column round
Appending data for column s6p5_sr
Appending data for column s6p5_source
Appending data for column s6p5_q1
Appending data for column s6p5_q2
Appending data for column s6p5_q3
Appending data for column s6p5_q4
Appending data for column P_ID
Appen

In [7]:
# Rename columns for the merged file (if needed)
rename_mapping = {
     'hid': 'HID',
    'round': 'Survey_Round',
    's6p5_sr': 'OI_SrNo',
    's6p5_source': 'OI_Src',
    's6p5_q1': 'OI_HH_ValRec',
    's6p5_q2': 'OI_Freceipts',
    's6p5_q3': 'OI_HH_ValRec_M',
    's6p5_q4': 'OI_ValT'
}

merged_df.rename(columns=rename_mapping, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_Section_6_part_6.csv', index=False)

